In [ ]:
import pandas as pd
from pathlib import Path
import os
import joblib
import numpy as np
from mapie.metrics import (
    classification_coverage_score,
    classification_mean_width_score
)
import re
import matplotlib.pyplot as plt
from utils.model_production_data_processing_utils import cluster_with_min_size
import umap

root = Path(os.getcwd()).parent

In [ ]:
from model_production_main import load_and_preprocess_data, prepare_features
from utils.model_production_data_processing_utils import compute_threshold_kmeans, build_X_s, build_umap_windows_by_suffix

In [ ]:
df25 = 

In [ ]:
df3 = pd.read_csv(root /"data/DATA_SPCI_ng_24.csv")

In [ ]:
[c for c in df3.columns if c.endswith('grade')]

In [ ]:
cols = df3.drop(columns=["email", "source", "clusters"]).columns.to_series()
w1 = 3
suffixes = cols.apply(lambda x: x.split("_")[1])
ordered_suffixes = suffixes.unique()
prefixes2 = (
    df3.drop(columns=["email"])
    .columns.str.split("_")
    .str[:2]
    .str.join("_")
    .unique()[w1 - 1 :][::-1]
)
change_points = [
    i for i in range(len(cols) - 1) if suffixes[i] != suffixes[i + 1]
][::-1]

In [ ]:
mark_cols = [c for c in df3.columns if c.endswith("mark")]
prefixes = list(dict.fromkeys(c.rsplit("_",1)[0] for c in mark_cols))
dyn_cols = [
        col for col in df3.columns
        if any(col.startswith(pref) for pref in prefixes[:11])
        ]
dyn_cols

In [ ]:
df_ = pd.read_csv(root / "data/DATA.csv")
df = df_.copy()
nb_nan_par_ligne = df.isna().sum(axis=1)
df0 = df[nb_nan_par_ligne < 495]

In [ ]:
df0.shape

In [ ]:
df0

In [ ]:
dfcpool = df0[[c for c in df0.columns if c.startswith("B-CPE-100")]]
pat = re.compile(r"B-CPE-100_cpoolday\d+_\d{2} - task\d+_passed")
cols_keep = [c for c in dfcpool.columns if not pat.match(c)]
dfcpool_mark = dfcpool[cols_keep]
X = dfcpool_mark.fillna(0)
ta = [c for c in X.columns if c.endswith('passed')]
X_bin = X[ta]


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
reducer = umap.UMAP(
    n_neighbors=8,          # ↓ pour plus local
    min_dist=0.25,          # ↑ étale un peu les amas
    spread=1.0,
    n_components=2,
    metric="hamming",
    random_state=42,
)
emb = reducer.fit_transform(X_bin)

K = 8
labels = KMeans(n_clusters=4, n_init="auto", random_state=42).fit_predict(emb)

In [ ]:
mark_cols = [c for c in X.columns if c.endswith("mark")]

In [ ]:
plt.figure(figsize=(7,6))
unique = np.unique(labels)
for lab in unique:
    mask = labels == lab
    plt.scatter(emb[mask, 0], emb[mask, 1], s=12, label=f"Cluster {lab}" if lab!=-1 else "Bruit (-1)", alpha=0.8)
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.title('title')
plt.legend(markerscale=1.5, fontsize=9, frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
emails = df0['email']
df_clusters = pd.DataFrame({
    "email": emails,
    "UMAP1": emb[:, 0],
    "UMAP2": emb[:, 1],
    "cluster": labels
})
df_clusters['mark'] = df0[mark_cols].mean(axis=1)

In [ ]:
import sqlite3

db_path = root / "data/clusters_umap.db"
table_name = "clusters"
conn = sqlite3.connect(db_path)
df_clusters.to_sql(table_name, conn, if_exists="replace", index=False)

# (Optionnel) index SQL pour accélérer les requêtes par email
conn.execute(f"CREATE INDEX IF NOT EXISTS idx_{table_name}_email ON {table_name}(email);")
conn.commit()

In [ ]:
from IPython.display import FileLink

df_clusters.to_csv("/data/DATA_proj.csv", index=False, encoding="utf-8")
FileLink("/data/DATA_proj.csv")

In [ ]:
df1 = load_and_preprocess_data(root / "data/DATA.csv", 24)

In [ ]:
def SPCI_lg_to_set(G, threshold):
    L, U = G
    if U < threshold:
        return [1]
    elif L > threshold:
        return [0]
    else:
        return [0,1]

In [ ]:
mod = joblib.load(root / "models" / "models_SPCI_lg_24.joblib")
mod.keys()

In [ ]:

mod_SPCI = joblib.load(root / "models" / "models_SPCI_lg_24.joblib")
mod_comb = joblib.load(root / "models" / "models_comb1_24.joblib")
mod_CP = joblib.load(root / "models" / "models_clustering_24.joblib")

In [ ]:
mod_comb.keys()

In [ ]:
y_true_arr = np.asarray(y_true).ravel().astype(int)
alpha = 0.1
max_N = 22

for base_model in ['GB']:
    for n in tqdm(range(1, max_N)):
        key = (base_model, n, "vanilla")
        X_CP = build_X_s(df2.fillna(0), prefixes, static_cols, n)
        model_CP = mod_CP[key]
        yp_van, yps_van = model_CP.predict(X_CP, alpha=alpha)  # partition=df2['clusters'])
        pset_van = yps_van[:, :, 0].astype(bool)
        cov = classification_coverage_score(y_true_arr, pset_van)
        wid = classification_mean_width_score(pset_van)
        covs[(base_model, n)] = cov
        wids[(base_model, n)] = wid
        if n <= w2:
            print(f"n={n:2d} | MCP: cov={cov:.3f}, wid={wid:.3f} | ")
        # Prédictions SPCI et combinées pour n > w2
        if n > w2: ## BELEK INDEX
            # Prédictions SPCI
            model_SPCI = mod_SPCI[n]
            X_SPCI = X_arr[n - w2]  # Ajustement de l'index comme dans train_combined_models
            intervals = np.array([model_SPCI.predict_interval(x) for x in X_SPCI], dtype=float)
            pset_spci = to_bool2_from_spci_intervals(intervals, threshold)
            cov_spci = classification_coverage_score(y_true_arr, pset_spci)
            wid_spci = classification_mean_width_score(pset_spci)
            covs_spci[(base_model, n)] = cov_spci
            wids_spci[(base_model, n)] = wid_spci
            
            # Prédictions combinées avec le modèle gate
            gate_model = mod_comb[(base_model, n)]
            
            # Calcul des features pour le gate (comme dans train_combined_models)
            w_cls = pset_van.sum(axis=1)  # Largeur des prédictions MCP
            w_spc = pset_spci.sum(axis=1)  # Largeur des prédictions SPCI
            diff = w_cls - w_spc
            
            # Construction de X_gate avec les mêmes features que lors de l'entraînement
            meta_features = np.column_stack([w_cls, w_spc, diff])
            X_gate = np.hstack([X_CP, meta_features])
            
            # Prédiction du gate : 0=MCP, 1=SPCI, 2=union
            gate_preds = gate_model.predict(X_gate)
            
            # Construction des prediction sets combinés
            pset_combined = np.zeros_like(pset_van, dtype=bool)
            
            for i in range(len(gate_preds)):
                if gate_preds[i] == 0:  # Utiliser MCP
                    pset_combined[i] = pset_van[i]
                elif gate_preds[i] == 1:  # Utiliser SPCI
                    pset_combined[i] = pset_spci[i]
                else:  # gate_preds[i] == 2, utiliser l'union
                    pset_combined[i] = pset_van[i] | pset_spci[i]
            
            # Calcul des métriques pour le modèle combiné
            cov_comb = classification_coverage_score(y_true_arr, pset_combined)
            wid_comb = classification_mean_width_score(pset_combined)
            covs_comb[(base_model, n)] = cov_comb
            wids_comb[(base_model, n)] = wid_comb
            
            # Affichage optionnel des résultats
            print(f"n={n:2d} | MCP: cov={cov:.3f}, wid={wid:.3f} | "
                  f"SPCI: cov={cov_spci:.3f}, wid={wid_spci:.3f} | "
                  f"Comb: cov={cov_comb:.3f}, wid={wid_comb:.3f}")
            
            # Statistiques sur les décisions du gate
            gate_choices, gate_counts = np.unique(gate_preds, return_counts=True)

In [ ]:
intervals

In [ ]:
covs_spci

In [ ]:
covs_comb

In [ ]:
mod_comb[('GB', 15)].n_features_in_ 

In [ ]:
mod_CP.keys()

In [ ]:
mod_SPCI.keys()

In [ ]:
mod_comb.keys()

In [ ]:
k = 16
print(build_X_s(df2.fillna(0), prefixes, static_cols, k).shape[1], mod_comb[('GB', k)].n_features_in_ )

In [ ]:
# df1 = df1.set_index('email')
df_tr_pred =  pd.merge(df0, preds_GB, on="email", how="inner")

In [ ]:
df_tr_pred

In [ ]:

from IPython.display import FileLink

df_tr_pred.to_csv("/data/DATA_2024_pred.csv", index=False, encoding="utf-8")
FileLink("/data/DATA_2024_pred.csv")

In [ ]:
preds_GB.head(20)
# preds_all.head()

In [ ]:
covs

In [ ]:
wids

In [ ]:
df3 = pd.read_csv(root / "data/DATA_SPCI_ng_24.csv")

In [ ]:
import numpy as np
from utils.models_production_utils import build_X_s

def gate_predict_minimal(
    dataframe, X_arr, n, base_model,
    models_c_ng, models_lg, models_comb,
    threshold, w2, prefixes, static_cols,
    alpha=0.05, partition=None  # partition=df['clusters'] si Mondrian
):
    """
    Renvoie:
      - p_final: bool array (n_samples, 2)  -> p-set final (après gate)
      - choice: int array (0=MCP, 1=SPCI, 2=union)
      - y_hat: int array -> 0/1 si singleton, -1 si ambigu (union des deux)
    """
    # 1) Features pour chaque “branche”
    X_CP = build_X_s(dataframe, prefixes, static_cols, n)      # mêmes colonnes/ordre qu’à l’entraînement
    idx_spci = n - w2
    X_SPCI = X_arr[idx_spci]

    # 2) Récupérer les modèles
    key_mcp = (base_model, n, "vanilla")  # ou "mondrian" si vous avez entraîné comme tel
    model_mcp = models_c_ng[key_mcp]
    model_spc = models_lg[n]
    gate = models_comb[(base_model, n)]

    # 3) p-sets MCP via MAPIE
    if partition is None:
        y_pred_mcp_gate, yps_mcp_gate = model_mcp.predict(X_CP, alpha=alpha)
    else:
        y_pred_mcp_gate, yps_mcp_gate = model_mcp.predict(X_CP, alpha=alpha, partition=partition)
    p_mcp = yps_mcp_gate[:, :, 0].astype(bool)  # (n_samples, 2)

    # 4) p-sets SPCI à partir des intervalles [L,U] et du threshold
    intervals = np.array([model_spc.predict_interval(x) for x in X_SPCI], dtype=float)
    L_cal = intervals[:, 0]
    U_cal = intervals[:, 1]
    p_spc = np.zeros_like(p_mcp, dtype=bool)
    p_spc[threshold < L_cal, 0] = True
    p_spc[threshold > U_cal, 1] = True
    amb = ~( (threshold < L_cal) | (threshold > U_cal) )
    p_spc[amb, :] = True  # ambigu → {0,1}

    # 5) Features pour la gate (mêmes que training): X_CP + [w_cls, w_spc, diff]
    w_cls = p_mcp.sum(axis=1)
    w_spc = p_spc.sum(axis=1)
    diff = w_cls - w_spc
    X_gate = np.hstack([X_CP, w_cls.reshape(-1,1), w_spc.reshape(-1,1), diff.reshape(-1,1)])

    # 6) Décision de la gate: 0=MCP, 1=SPCI, 2=union
    choice = gate.predict(X_gate)

    # 7) Composer le p-set final selon la gate
    p_final = np.empty_like(p_mcp, dtype=bool)
    use_mcp = (choice == 0)
    use_spc = (choice == 1)
    use_uni = (choice == 2)
    p_final[use_mcp] = p_mcp[use_mcp]
    p_final[use_spc] = p_spc[use_spc]
    p_final[use_uni] = (p_mcp[use_uni] | p_spc[use_uni])

    # 8) Étiquette ponctuelle minimale: 0/1 si singleton, sinon -1 (ambigu)
    singletons = (p_final.sum(axis=1) == 1)
    y_hat = np.where(singletons, p_final.argmax(axis=1), -1)

    return {
        "p_final": p_final,   # bools shape (n_samples, 2)
        "choice": choice,     # 0/1/2
        "y_hat": y_hat        # 0/1 ou -1 si ambigu
    }


In [ ]:
root = Path(os.getcwd()).parent

In [ ]:
obj = joblib.load(root / "models" / "models_clustering_24.joblib")

In [ ]:
obj2 = joblib.load(root / "models" / "models_clustering_SPCI_ng_24.joblib")

In [ ]:
type(obj2)

In [ ]:
obj2.keys()

In [ ]:
df3 = pd.read_csv(root / "data/DATA_SPCI_ng_24.csv")
df3

In [ ]:
df = pd.read_csv(root / "data/DATA.csv")
nb_nan_par_ligne = df.isna().sum(axis=1)

df = df[nb_nan_par_ligne < 495]

In [ ]:
len(df)

In [ ]:
def build_X_s(df_sub: pd.DataFrame, prefixes: list, static_cols: list, n: int) -> np.ndarray:
    # on garde student_id + les n premiers items
    dyn_cols = [
    col for col in df_sub.columns
    if any(col.startswith(pref) for pref in prefixes[:n])
    ]
    keep = ["email"] + static_cols + dyn_cols
    return df_sub[keep].set_index("email").values

In [ ]:
mark_cols = [c for c in df.columns if c.endswith("mark")]
prefixes = list(dict.fromkeys(c.rsplit("_",1)[0] for c in mark_cols))
static_cols = []

In [ ]:
X = build_X_s(df.fillna(0), prefixes, static_cols, 3)

In [ ]:
dfcpool = df[[c for c in df.columns if c.startswith("B-CPE-100")]]
pat = re.compile(r"B-CPE-100_cpoolday\d+_\d{2} - task\d+_passed")
cols_keep = [c for c in dfcpool.columns if not pat.match(c)]
dfcpool_mark = dfcpool[cols_keep]
X_pool = dfcpool_mark.fillna(0)

In [ ]:
df2, info = cluster_with_min_size(
    df, X_pool, n_clusters=4, min_cluster_size=50, random_state=42)

In [ ]:
res = []
for n in range(1, 16):
    mod = obj2[('GB', n, 'vanilla')]
    X = build_X_s(df2.fillna(0), prefixes, static_cols, n)
    yp_van, yps_van = mod.predict(X, alpha=0.1) # partition=df2['clusters'])
    pset_van = yps_van[:, :, 0]
    print(classification_mean_width_score(pset_van))
    res.append(classification_mean_width_score(pset_van))
print("moy", np.mean(res))

In [ ]:
redoublants = [
    "vondandamo",
    "beugre lupetu kamba",
    "abdallah",
    "ramirez",
    "essonghe",
    "haye",
    "mezou",
    "rocagel",
    "rakrouki",
    "tchakal",
    "galloo",
    "fougere",
    "fillon",
    "henine",
    "angibaud",
    "marechal",
    "frotin",
    "eyraud"
]

df0.head()

In [ ]:
from IPython.display import FileLink

mark_cols = [c for c in df0.columns if c.endswith("mark")]
df_redoublants = df0[df0['email'].str.split(".").str[1].str.split("@").str[0].isin(redoublants)][mark_cols]

df_redoublants.to_csv(root /"/data/grades_redoublants.csv", index=False, encoding="utf-8")
FileLink("/data/grades_redoublants.csv")

In [ ]:
for redoublant in redoublants:
    for eleve in df0['email'].str.split(".").str[1].str.split("@").str[0]:
        if redoublant == eleve:
            print(eleve)